# Spiral Resonance — Interactive Notebook

> The spiral is the hum made visible — micro oscillators aligning into macro coherence.

This notebook mirrors `sims/spiral_resonance.py` but renders the animation **inline**.

**Outputs**:
- Inline animation preview (JS/HTML when supported)
- Optional GIF saved to `sims/figures/spiral_resonance.gif`

Run the cells top→bottom. No seaborn; plain matplotlib.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, Image, display
import os

plt.rcParams['figure.figsize'] = (6,6)

In [ ]:
# Parameters (you can tweak and rerun)
N = 200           # number of oscillators
steps = 400       # time steps
K = 1.2           # coupling strength (↑ → faster lock)
dt = 0.05         # time step
seed = 0          # randomness control

np.random.seed(seed)

# Initial phases & natural frequencies
theta = np.random.uniform(0, 2*np.pi, N)
omega = np.random.normal(0.0, 0.5, N)

# History for animation
history = []

def step():
    global theta
",
    # Kuramoto-like update (mean-field coupling)
    coupling = K * np.imag(np.exp(1j*(theta[:,None]-theta[None,:]))).mean(axis=1)
    theta = theta + (omega + coupling) * dt
    history.append(theta.copy())

# Run simulation
for _ in range(steps):
    step()

len(history)

In [ ]:
# Convert phase history to spiral coordinates over time
x_hist, y_hist = [], []
for t_idx, thetas in enumerate(history):
    radius = 1 + t_idx/steps * 3.0   # outward spiral radius growth
    x = radius * np.cos(thetas)
    y = radius * np.sin(thetas)
    x_hist.append(x)
    y_hist.append(y)

len(x_hist), len(y_hist)

In [ ]:
# Build animation
fig, ax = plt.subplots()
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_aspect('equal')
ax.set_title('Spiral Resonance (micro → macro coherence)')
scat = ax.scatter([], [], s=10, alpha=0.7)

def animate(i):
    scat.set_offsets(np.c_[x_hist[i], y_hist[i]])
    return scat,

ani = animation.FuncAnimation(fig, animate, frames=len(x_hist), interval=50, blit=True)
plt.close(fig)  # prevent duplicate static rendering

### Inline Preview
Below, we try JS/HTML first (`to_jshtml()`), which plays directly in the notebook.
If that isn't supported, we fall back to generating a GIF and showing it inline.

In [ ]:
# Try HTML preview (no external writer needed)
try:
    html = ani.to_jshtml()
    display(HTML(html))
    used_jshtml = True
except Exception as e:
    print('JS/HTML preview not available, falling back to GIF:', e)
    used_jshtml = False

### Optional: Save GIF (and preview)
This requires the `pillow` package. The GIF will be stored under `sims/figures/`.

In [ ]:
if not used_jshtml:
    os.makedirs('sims/figures', exist_ok=True)
    out_path = 'sims/figures/spiral_resonance.gif'
    try:
        ani.save(out_path, writer='pillow', fps=20)
        display(Image(filename=out_path))
        print('Saved:', out_path)
    except Exception as e:
        print('Could not save GIF. Install pillow or run the .py script instead. Error:', e)
else:
    print('HTML preview displayed above. (You can still save a GIF by forcing the cell above to run with used_jshtml=False.)')

## Notes
- Adjust `K` (coupling) to see faster/slower coherence.
- `steps` controls animation length; `N` controls node count.
- The outward radius encodes the **micro → macro** growth while phases align.
- To regenerate as a file without this notebook, run: `python sims/spiral_resonance.py`.